In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,311 kB]
Get:14 http://security.ubuntu.com/ubunt

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-13 22:14:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2023-03-13 22:14:12 (1.25 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-MM-dd")
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-08-31 00:00:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Get the number of rows in the DataFrame.
print(f'number of rows in the Data Frame = {df.count()}')

number of rows in the Data Frame = 2643619


In [ ]:
# Drop null values, by default is any which means will drop any rows which has a NA
print((df.count(),len(df.columns)))
df2 = df.dropna()
df3 = df2.dropDuplicates()
print((df3.count(),len(df3.columns)))
df3.show()


(2643619, 15)
(2643241, 15)
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   26879429|R1015HJHGZUJVG|B00076HSUC|     421502374|Cholodin-FEL For ...|    Pet Products|          5|            1|          1|   N|                Y|      Helps Old Cats|Cholodin is a gre...|2013-08-11 00:00:00|
|         US|   49345442|R1023YMO9XR93S|B00171IVHA|      251

In [ ]:
df3.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

# Transform the Data

## Create the "review_id_table".

In [ ]:
# from pyspark.sql.functions import to_date
from pyspark.sql.functions import *
from pyspark.sql.types import *
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table_df = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df2 = review_id_table_df.withColumn("review_date",to_date(col("review_date")))
# review_id_table_df.show(truncate=False)

In [ ]:
type(review_id_table_df2)

pyspark.sql.dataframe.DataFrame

In [ ]:
review_id_table_df2.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_table_df2.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1015HJHGZUJVG|   26879429|B00076HSUC|     421502374| 2013-08-11|
|R1023YMO9XR93S|   49345442|B00171IVHA|      25146499| 2014-12-02|
|R10267J8PBWEVQ|   48576217|B0024E0QHC|     857059897| 2014-06-25|
|R103326H6Z3CTG|   34624253|B000UZNLGA|     144491764| 2011-11-12|
| R1035J76P280Y|   48425022|B003IRJTCC|     996646406| 2010-12-22|
| R103IP0FCBH8I|   14356933|B0002AQPKW|     152333311| 2011-12-12|
|R103WDQ4IDVIUV|   14302088|B00E8LYJS6|     394306248| 2014-12-15|
|R103WQ2GZQLH6T|   13853268|B006MUQO0M|      84171195| 2015-06-26|
|R104K30IR1GWUD|   27083213|B008V68GKG|     915709880| 2013-07-18|
|R104LLIZQ8O3DW|   18411324|B00ETS9LZ4|     259591253| 2014-06-05|
|R104OCMU5PF85T|    2654230|B008CO4H8C|     648340511| 2015-03-03|
|R104QASUJNTTNP|   21635417|B0056J2PWG|      26863086| 2014-04

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df2 = df3.select(["product_id", "product_title"])
products_df = products_df2.dropDuplicates()
print((products_df.count(),len(products_df.columns)))
products_df.show()

(239307, 2)
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

from pyspark.sql.functions import desc,count

customers_df= df3.groupBy('customer_id').agg(count("customer_id").alias("customer_count"))
customers_df.show()
# df4 = df3.groupBy('customer_id').agg({'customer_id':'count'}).orderBy(desc('count(customer_id)'))
# df5 = df4.withColumnRenamed("count(customer_id)","customer_count")
# customers_df = df5.withColumn("customer_count",to_int(col("customer_count")))
# customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21105356|             5|
|   29425552|             2|
|   17041448|             5|
|   37882821|             1|
|   42532905|             1|
|   18759528|             1|
|   19172324|             4|
|   51474275|             2|
|   27911347|             9|
|   22602819|             2|
|    5658913|            34|
|   14691760|             2|
|   13645331|             1|
|   14139374|             1|
|   14397122|             4|
|   10850500|             1|
|   47233188|             6|
|   24215961|             1|
|   16468643|             1|
|   18630443|             8|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



## Create the "vine_table".

In [ ]:
df3.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1015HJHGZUJVG|          5|            1|          1|   N|
|R1023YMO9XR93S|          2|            0|          0|   N|
|R10267J8PBWEVQ|          5|            0|          0|   N|
|R103326H6Z3CTG|          5|            1|          1|   N|
| R1035J76P280Y|          5|            6|          7|   N|
| R103IP0FCBH8I|          5|            1|          1|   N|
|R103WDQ4IDVIUV|          5|            0|          0|   N|
|R103WQ2GZQLH6T|          5|            0|          0|   N|
|R104K30IR1GWUD|          5|            0|          0|   N|
|R104LLIZQ8O3DW|          5|            0|          0|   N|
|R104OCMU5PF85T|          2|           25|         25|   N|
|R104QASUJNTTNP|          5|            0|          0|   N|
|R1058KF5BARAYI|          5|            0|          0|   N|
|R105G5FW41EYXT|          3|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", "password": "123postgres456", "driver":"org.postgresql.Driver"}

# # Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/Employee_stu_etl_rds"
# # jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/Employee_stu_etl_rds"
# config = {"user":"postgres", 
#           "password": "postgres", 
#           "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_table_df2.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)